# Final Project Repository

In [16]:
library(tidyverse)
library(repr)
library(infer)
library(cowplot)
library(broom)
library(modelr)
library(pROC)
library(rsample)
library(caret)
install.packages("glmnet")
library(glmnet)


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loaded glmnet 4.1-8



### Introduction


### Methods and Results

### Discussion

In [39]:
set.seed(3)
email <- read_csv(url("https://www.openintro.org/data/csv/email.csv"))
email <- email |>
    select(-time, -viagra)
email <- email |>
    mutate(across(c(spam, to_multiple, from, sent_email, winner, format, re_subj, exclaim_subj, urgent_subj, number), as.factor))

email_split <- initial_split(email, prop = 0.7, strata = spam)

email_training <- training(email_split)

email_testing <- testing(email_split)

matrix_X_train <- 
    model.matrix(object = spam ~ .,
                 data = email_training)[,-1]

matrix_Y_train <- 
    as.matrix(email_training$spam, ncol = 1)

Rows: 3921 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (2): winner, number
dbl  (18): spam, to_multiple, from, cc, sent_email, image, attach, dollar, i...
dttm  (1): time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [40]:
set.seed(4)
email_cv_lambda_LASSO <- cv.glmnet(
    x = matrix_X_train, y = matrix_Y_train,
    alpha = 1,
    family = "binomial",
    type.measure = "auc",
    nfolds = 5
    )

lambda_min_AUC_LASSO <- round(email_cv_lambda_LASSO$lambda.min, 4)

email_LASSO_min_AUC <- 
    glmnet(
        x = matrix_X_train, 
        y = matrix_Y_train,
        alpha = 1,
        family = "binomial",
        lambda = lambda_min_AUC_LASSO)

coef(email_LASSO_min_AUC)



20 x 1 sparse Matrix of class "dgCMatrix"
                        s0
(Intercept)    3.360152049
to_multiple1  -2.160884257
from1         -3.662802449
cc             0.026470888
sent_email1   -4.582798080
image         -1.169070794
attach         0.464846082
dollar        -0.065503955
winneryes      1.905970532
inherit        0.291403655
password      -0.609556804
num_char       .          
line_breaks   -0.002686705
format1       -0.622305227
re_subj1      -1.789314087
exclaim_subj1  .          
urgent_subj1   3.556141846
exclaim_mess   0.008256679
numbernone     0.161046645
numbersmall   -0.888828627

### References